In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline  

import torch
import torch.nn as nn

import arm_experiments_lib as arm_lib

import torch.optim as optim

from copy import deepcopy

from torch.distributions import Categorical

import itertools

In [2]:
softmax = nn.Softmax(dim = 0)

sigmoid = nn.Sigmoid()

In [3]:
np.random.seed(454)
_ = torch.manual_seed(454)

In [4]:
# fixed parameters
d = 3
# p0 = torch.rand(d)
p0 = torch.Tensor([0.51, 0.51, 0.51])
print('p0: ', p0, '\n')

print('sum(p0^2): ', torch.sum(p0**2))
print('sum((1 - p0)^2): ', torch.sum((1 - p0)**2), '\n')

# the optima
x_optimal = torch.argmin(torch.Tensor([torch.sum(p0**2), torch.sum((1 - p0)**2)]))

optimal_loss = torch.min(torch.Tensor([torch.sum(p0**2), torch.sum((1 - p0)**2)]))

print('optimal loss: ', optimal_loss)
print('optimal x: ', x_optimal.numpy())

p0:  tensor([ 0.5100,  0.5100,  0.5100]) 

sum(p0^2):  tensor(0.7803)
sum((1 - p0)^2):  tensor(0.7203) 

optimal loss:  tensor(0.7203)
optimal x:  1


In [5]:
# random init for phi
phi0 = torch.rand(1, requires_grad=True)
phi0.requires_grad_(True)
print('init: ', phi0)

init:  tensor([ 0.1876])


In [6]:
e_b = sigmoid(phi0)

In [7]:
probs, draw_array = arm_lib.get_all_probs(e_b, d)

In [8]:
draw_array

[tensor([ 0.,  0.,  0.]),
 tensor([ 0.,  0.,  1.]),
 tensor([ 0.,  1.,  0.]),
 tensor([ 0.,  1.,  1.]),
 tensor([ 1.,  0.,  0.]),
 tensor([ 1.,  0.,  1.]),
 tensor([ 1.,  1.,  0.]),
 tensor([ 1.,  1.,  1.])]

In [9]:
probs

tensor([ 0.0931,  0.1123,  0.1123,  0.1355,  0.1123,  0.1355,  0.1355,
         0.1635])

In [10]:
for i in range(d + 1): 
    print(e_b**i * (1 - e_b)**(d - i))

tensor(1.00000e-02 *
       [ 9.3105])
tensor([ 0.1123])
tensor([ 0.1355])
tensor([ 0.1635])


In [11]:
losses_array = arm_lib.get_losses_from_draw_array(draw_array, p0)

In [12]:
losses_array

tensor([ 0.7803,  0.7603,  0.7603,  0.7403,  0.7603,  0.7403,  0.7403,
         0.7203])

In [13]:
# get probabilities for all 2^d combinations
probs, draw_array = arm_lib.get_all_probs(e_b, d)

# get losses for all 2^d combinations
losses_array = arm_lib.get_losses_from_draw_array(draw_array, p0)

In [14]:
print(probs)

tensor([ 0.0931,  0.1123,  0.1123,  0.1355,  0.1123,  0.1355,  0.1355,
         0.1635])


In [21]:
num_reinforced = 2

In [22]:
# sample from conditional probabiltiies
z_sample, sampled_z_domain, sampled_weight, \
        unsampled_z_domain, unsampled_weight = \
            arm_lib.sample_class_weights(probs, num_reinforced)

print(sampled_z_domain)
print(unsampled_z_domain)

tensor([ 0,  2])
tensor([ 1,  3,  4,  5,  6,  7])


# Testing for sample class_weights function

In [23]:
probs = torch.Tensor([0.1, 0.4, 0.3, 0.05, 0.15])
num_reinforced = 2

In [24]:
# sample from conditional probabiltiies
z_sample, sampled_z_domain, sampled_weight, \
        unsampled_z_domain, unsampled_weight = \
            arm_lib.sample_class_weights(probs, num_reinforced)

In [25]:
print(sampled_z_domain)
print(unsampled_z_domain)

tensor([ 3,  0])
tensor([ 1,  2,  4])


In [26]:
print(sampled_weight)
print(unsampled_weight)

tensor([ 0.0500,  0.1000])
tensor([ 0.4000,  0.3000,  0.1500])


In [27]:
z_sample_array = np.zeros(10000)
for i in range(len(z_sample_array)): 
    z_sample = arm_lib.sample_class_weights(probs, num_reinforced)[0]
    z_sample_array[i] = z_sample

In [28]:
propn0 = np.sum(z_sample_array == 0) / len(z_sample_array)
propn3 = np.sum(z_sample_array == 3) / len(z_sample_array)
propn4 = np.sum(z_sample_array == 4) / len(z_sample_array)

In [29]:
print(propn0) 
print(propn3)
print(propn4)

0.6686
0.3314
0.0
